In [134]:
%run helper.ipynb

In [64]:
df=pd.read_csv(all_data)

In [135]:
data = DataAnalyzer(all_data,df=df)

In [111]:
data.load_stats()

In [66]:
dir(data)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'df',
 'features',
 'features_with_nan',
 'fill_missing',
 'find_missing_years',
 'get_features',
 'len',
 'load_stats',
 'src_file',
 'total_features']

In [136]:
data.len, data.total_features

(1799436, 594)

In [116]:
data.nans['Population']

1004758

In [117]:
data.zeros['Population']

289

In [137]:
data.get_features("beverage")

['Alcoholic_Beverage_Tax', 'Alcoholic_Beverage_Lic']

In [138]:
len(data.features_with_nan)

59

In [140]:
# data.negatives['Total_Revenue']

In [143]:
for f in data.features_with_nan:
    print(data.find_missing_years(f))


YearofData: 1094304(60.8137%) missing values which are for years [2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990, 1989, 1988, 1987, 1986, 1985, 1984, 1983, 1982, 1981, 1980, 1979, 1978, 1977, 1976, 1975, 1974, 1973, 1972, 1971, 1970, 1967]
IDChanged: 374515(20.8129%) missing values which are for years [2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007]
Name: 1(0.0001%) missing values which are for years [2014]
FIPS_Code_State: 569(0.0316%) missing values which are for years [2014, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990, 1989, 1988, 1987, 1986, 1985, 1984, 1983, 1982, 1981, 1980, 1979, 1978, 1977, 1976, 1975, 1974, 1973, 1972, 1971, 1970, 1967]
FIPS_County: 766557(42.5998%) missing values which are for years [2016, 2015, 2014, 2013, 2012, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2

In [144]:
t = data.df.loc[data.df['State_IGR_Gen_Sup'] != data.df['State_IGR_Gen_Sup']]

In [145]:
t['Type_Code'].value_counts()

5    13464
4     4262
2      617
1      317
3      154
0       50
Name: Type_Code, dtype: int64

In [146]:
data.fill_missing('State_IGR_Gen_Sup')

In [152]:
data.df['State_IGR_Gen_Sup'].isna().sum()

0

In [18]:
t1 = data.df.loc[data.df['State_IGR_Gen_Sup'] != data.df['State_IGR_Gen_Sup']]

In [20]:
t1['Type_Code'].value_counts()

Series([], Name: Type_Code, dtype: int64)

In [154]:
missing_feature_map['Motor_Vehicle_License_Total']

{'op': 'ADD',
 'src_features': ['Motor_Vehicle_License', 'Motor_Veh_Oper_License']}

**TODO : make it work for below features**
```
Motor_Vehicle_License_Total
State_IGR_Gen_Sup
Chg_Total_Elem_Education
Chg_Highways
Prop_Sale_Total
Total_Other_Capital_Outlays
General_Capital_Outlay_Other
Air_Trans_Current_Exp
Misc_Com_Activ_Current_Exp
Educ_NEC_Current_Exp
Fin_Admin_IG_Local_Govts
Judicial_Current_Exp
Health_Current_Exp
Total_Highways_Current_Exp
Toll_Hwy_Current_Exp
Welf_NEC_Current_Exp
Sewerage_Current_Exp
SW_Mgmt_Current_Exp
Liquor_Stores_Current_Exp

```


In [98]:
d = {}
feature_current_list =['Total_Highways_Current_Exp']
for f in feature_current_list:
    cat = f.split("Current")[0]
    related = get_features(cat)
    # Case 1 : (Direct - capital outlay)
    interest = "{}Inter_Exp".format(cat)
    direct = "{}Direct_Exp".format(cat)
    total = "{}Total_Exp".format(cat)
    print("{}:{}".format(f,related))
#     print("***",related,s)
    if  interest in related:
        f1 = "{}Total_Exp".format(cat)
        f3 = "{}Cap_Outlay".format(cat)
        d[f] = {'op' : "SUBSTRACT", "src_features" : [f1,interest,f3]}
    elif  direct in related:
        cap_ol = "{}Cap_Outlay".format(cat)
        d[f] = {'op' : "SUBSTRACT", "src_features" : [direct,cap_ol]}
    elif total in related:
        cap_ol = "{}Cap_Out".format(cat)
        d[f] = {'op' : "SUBSTRACT", "src_features" : [total,cap_ol]}
        
print(d)

Total_Highways_Current_Exp:['Total_Highways_Tot_Exp', 'Total_Highways_Dir_Exp', 'Total_Highways_Cap_Out', 'Total_Highways_Current_Exp', 'Total_Highways_Construct']
{}


In [73]:
len(data.features_with_nan),len(missing_feature_map)

(59, 46)

In [91]:
for key, val in missing_feature_map.items():
    if key not in d:
        print(key)
#         val1 = d[key]
#         if val['op'] != val1['op']:
#             print(key, val,val1)
                    
#         if len(val['src_features']) != len(val1['src_features']):
#             print(val,val1)
# #         if val['src_features'] != val1['src_features']:
#             print("Same src_features length but different features")
#             print(val,val1)
            

Motor_Vehicle_License_Total
State_IGR_Gen_Sup
Chg_Total_Elem_Education
Chg_Highways
Prop_Sale_Total
Total_Other_Capital_Outlays
General_Capital_Outlay_Other
Air_Trans_Current_Exp
Misc_Com_Activ_Current_Exp
Educ_NEC_Current_Exp
Fin_Admin_IG_Local_Govts
Judicial_Current_Exp
Health_Current_Exp
Total_Highways_Current_Exp
Toll_Hwy_Current_Exp
Welf_NEC_Current_Exp
Sewerage_Current_Exp
SW_Mgmt_Current_Exp
Liquor_Stores_Current_Exp


In [58]:
t=df.isna().any().to_dict()

In [61]:
l = [ x for x in t.keys() if t[x]]

In [63]:
df['Prop_Sale_Total'].isna().sum()

0

In [17]:
before_filling={}
for key in missing_feature_map.keys():
    nan_count = data.df[key].isna().sum()
    details = data.df.loc[data.df[key] != data.df[key]]['Type_Code'].value_counts().to_dict()
    before_filling[key] = {'count' : nan_count,'datails':details}

In [18]:
before_filling

{'Motor_Vehicle_License_Total': {'count': 23239,
  'datails': {5: 13464, 4: 4246, 2: 3089, 1: 1433, 3: 1007}},
 'State_IGR_Gen_Sup': {'count': 18864,
  'datails': {5: 13464, 4: 4262, 2: 617, 1: 317, 3: 154, 0: 50}},
 'Chg_Total_Elem_Education': {'count': 11075,
  'datails': {4: 4265, 2: 3301, 1: 1725, 5: 999, 3: 744, 0: 41}},
 'Chg_Highways': {'count': 22803,
  'datails': {5: 13464, 4: 4220, 2: 2832, 1: 1374, 3: 912, 0: 1}},
 'Prop_Sale_Total': {'count': 17125,
  'datails': {5: 9848, 4: 3703, 2: 1902, 1: 939, 3: 730, 0: 3}},
 'Total_Other_Capital_Outlays': {'count': 2900,
  'datails': {4: 1558, 5: 614, 2: 373, 3: 178, 1: 177}},
 'General_Capital_Outlay_Other': {'count': 3573,
  'datails': {4: 2199, 5: 614, 2: 402, 3: 181, 1: 177}},
 'Air_Trans_Current_Exp': {'count': 22734,
  'datails': {5: 13464, 4: 4171, 2: 2617, 1: 1464, 3: 1011, 0: 7}},
 'Misc_Com_Activ_Current_Exp': {'count': 22944,
  'datails': {5: 13464, 4: 4216, 2: 2585, 1: 1892, 3: 749, 0: 38}},
 'Correct_Current_Exp': {'count

In [28]:
done=set()
for key in missing_feature_map.keys():
    print(key)
    if key not in done:
        data.fill_missing(key)
        done.update(key)

Motor_Vehicle_License_Total
State_IGR_Gen_Sup
Chg_Total_Elem_Education
Chg_Highways
Prop_Sale_Total
Total_Other_Capital_Outlays
General_Capital_Outlay_Other
Air_Trans_Current_Exp
Misc_Com_Activ_Current_Exp
Correct_Current_Exp
Elem_Educ_Current_Exp
Higher_Ed_Current_Exp
Educ_NEC_Current_Exp
Emp_Sec_Adm_Current_Exp
Fire_Prot_Current_Exp
Cen_Staff_Current_Exp
Gen_Pub_Bldg_Current_Exp


KeyError: 'Gen_Pub_Bldg_Cap_Outlay'

In [25]:
after_filling={}
for key in missing_feature_map.keys():
    nan_count = data.df[key].isna().sum()
    details = data.df.loc[data.df[key] != data.df[key]]['Type_Code'].value_counts().to_dict()
    after_filling[key] = {'count' : nan_count,'datails':details}

In [26]:
after_filling

{'Motor_Vehicle_License_Total': {'count': 0, 'datails': {}},
 'State_IGR_Gen_Sup': {'count': 0, 'datails': {}},
 'Chg_Total_Elem_Education': {'count': 0, 'datails': {}},
 'Chg_Highways': {'count': 0, 'datails': {}},
 'Prop_Sale_Total': {'count': 0, 'datails': {}},
 'Total_Other_Capital_Outlays': {'count': 0, 'datails': {}},
 'General_Capital_Outlay_Other': {'count': 0, 'datails': {}},
 'Air_Trans_Current_Exp': {'count': 0, 'datails': {}},
 'Misc_Com_Activ_Current_Exp': {'count': 0, 'datails': {}},
 'Correct_Current_Exp': {'count': 0, 'datails': {}},
 'Elem_Educ_Current_Exp': {'count': 0, 'datails': {}},
 'Higher_Ed_Current_Exp': {'count': 0, 'datails': {}},
 'Educ_NEC_Current_Exp': {'count': 0, 'datails': {}},
 'Emp_Sec_Adm_Current_Exp': {'count': 0, 'datails': {}},
 'Fire_Prot_Current_Exp': {'count': 19254,
  'datails': {5: 13464, 4: 4041, 1: 978, 2: 591, 3: 130, 0: 50}},
 'Cen_Staff_Current_Exp': {'count': 0, 'datails': {}},
 'Gen_Pub_Bldg_Current_Exp': {'count': 19741,
  'datails': 

In [94]:
for key,val in missing_feature_map.items():
    for f in val['src_features']:
        if f not in features:
            print(key,f)

Gen_Pub_Bldg_Current_Exp Gen_Pub_Bldg_Cap_Outlay
Total_Hospital_Current_Exp Total_Hospital_Cap_Outlay
Own_Hospital_Current_Exp Own_Hospital_Cap_Outlay


In [99]:
state = df.loc[df['Type_Code'] == 0]